Soma total de vendas por dia , ordens com status de 'Delivered' e estate = NY

In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
display( dbutils.fs.ls(silver_path) )

In [0]:
%python
silver_map = {
    #"tmp_silver_customer":      f"{silver_path}/customer/",
    "tmp_silver_orders":        f"{silver_path}/orders_enriched",
    #"tmp_silver_product":       f"{silver_path}/product/",

}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
df_sales_ny = spark.sql("""
    SELECT
        shipped_date,
        round(sum(total_sale), 2) AS total_sales
    FROM tmp_silver_orders
    WHERE state = 'NY'
      AND order_status_desc = 'Delivered'
      AND shipped_date IS NOT NULL
    GROUP BY shipped_date
""")

df_sales_ny.write \
    .mode('overwrite') \
    .format('delta') \
    .option('mergeSchema', 'true') \
    .save(f"{gold_path}/sales_ny")
